# Machine learning applied to HFRD flamelet database

In this notebook, we will train neural networks to replace the CVODE solver of CANTERA. We will use the databases generated in the *Flamelet_database_generation.ipynb* notebook.

In [ ]:
use_colab = False

## Google colab preparation

These lines are here to enable Colab running of the tools. We need to perform a git clone in order to have access to python scripts. This needs to be done at each runtime as the clone is lost. 

In [ ]:
import os

if use_colab:
    !git clone -b cost_course_exercices https://github.com/cmehl/ML_chem.git
    
    !pip install cantera

    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Create a folder in the root directory
    if not os.path.isdir("/content/drive/MyDrive/ML_chem_data"):
        !mkdir -p "/content/drive/MyDrive/ML_chem_data"
    else:
        print("Folder /content/drive/MyDrive/ML_chem_data already exists")

## Imports and options

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import time
import json
import joblib
import pickle
import numpy as np
import pandas as pd

import cantera as ct

import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme("notebook")

if use_colab:
    from ML_chem.chem_ai.cantera_runs_clustering import compute_nn_cantera_0D_homo, compute_nn_cantera_1D_prem, compute_nn_cantera_1D_diff
    from ML_chem.chem_ai.utils import get_molar_mass_atomic_matrix
    from ML_chem.chem_ai.utils import StandardScaler
else:
    from chem_ai.cantera_runs_clustering import compute_nn_cantera_0D_homo, compute_nn_cantera_1D_prem, compute_nn_cantera_1D_diff
    from chem_ai.utils import get_molar_mass_atomic_matrix
    from chem_ai.utils import StandardScaler

We set the default pytorch precision to double. It slows down a little bit the training but it is the usual standard for CFD reacting flows applications.

In [ ]:
torch.set_default_dtype(torch.float64)

We identify the device (CPU or GPU) available on the machine. This will be used by pytorch to identify the device on which to train and use the model:

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('Running on the GPU')
else:
  device = torch.device('cpu')
  print('Running on the CPU')

## Preliminary

### Parameters

In [ ]:
if use_colab:
    folder = "/content/drive/MyDrive/ML_chem_data/case_0D_highT"
else:
    folder = "./case_multi_test_case_flamelets" 

In [ ]:
with open(os.path.join(folder, "dtb_params.json"), "r") as file:
    dtb_params = json.load(file)

fuel = dtb_params["fuel"]
mech_file = dtb_params["mech_file"]
log_transform = dtb_params["log_transform"]
threshold = dtb_params["threshold"]
p = dtb_params["p"]
dt = dtb_params["dt"]
n_clusters = dtb_params["n_clusters"]


print(f"fuel = {fuel}")
print(f"mech_file = {mech_file}")
print(f"log_transform = {log_transform}")
print(f"threshold = {threshold}")
print(f"p = {p}")
print(f"dt = {dt}")
print(f"n_clusters = {n_clusters}")

### Data loading function

In [ ]:
def load_data(i_cluster):

    Xscaler = joblib.load(os.path.join(folder, f"processed_database_cluster/cluster_{i_cluster}", "Xscaler.pkl"))
    Yscaler = joblib.load(os.path.join(folder, f"processed_database_cluster/cluster_{i_cluster}", "Yscaler.pkl"))

    X_train = pd.read_csv(os.path.join(folder, f"processed_database_cluster/cluster_{i_cluster}","X_train.csv"))
    X_val = pd.read_csv(os.path.join(folder, f"processed_database_cluster/cluster_{i_cluster}","X_val.csv"))
    Y_train = pd.read_csv(os.path.join(folder, f"processed_database_cluster/cluster_{i_cluster}","Y_train.csv"))
    Y_val = pd.read_csv(os.path.join(folder, f"processed_database_cluster/cluster_{i_cluster}","Y_val.csv"))

    return Xscaler, Yscaler, X_train, X_val, Y_train, Y_val

We load the first cluster (necessarily present) to get some needed information for designing the models:

In [ ]:
_, _, X_train, _, Y_train, _ = load_data(0)

# Additional variable needed
Xcols = X_train.columns
Ycols = Y_train.columns

n_in = X_train.shape[1]
n_out = Y_train.shape[1]

### Elements conservation matrix

See notebook *0D_ann_learning.ipynb* to get details.

In [ ]:
gas = ct.Solution(mech_file)
A_element = get_molar_mass_atomic_matrix(gas.species_names, fuel, True)
print(A_element)

A_element = torch.tensor(A_element, dtype=torch.float64)
A_element = A_element.to(device)

## ANN models training

### Main training function

In [ ]:
def main_training_loop(X_train, X_val, Y_train, Y_val, loss_fn, optimizer, n_epochs, batch_size, model, log_transform, Xscaler_mean, Yscaler_mean, Xscaler_std, Yscaler_std):

    # Array to store the loss and validation loss
    loss_list = np.empty(n_epochs)
    val_loss_list = np.empty(n_epochs//10)

    # Array to store sum of mass fractions: mean, min and max
    stats_sum_yk = np.empty((n_epochs//10,3))

    # Array to store elements conservation: mean, min and max
    stats_A_elements = np.empty((n_epochs//10,4,3))

    epochs = np.arange(n_epochs)
    epochs_small = epochs[::10]

    for epoch in range(n_epochs):

        # Training parameters
        for i in range(0, len(X_train), batch_size):

            Xbatch = X_train[i:i+batch_size]
            y_pred = model(Xbatch)
            ybatch = Y_train[i:i+batch_size]
            loss = loss_fn(y_pred, ybatch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_list[epoch] = loss

        # Computing validation loss and mass conservation metric (only every 10 epochs as it is expensive)
        if epoch%10==0:
            model.eval()  # evaluation mode
            with torch.no_grad():

                # VALIDATION LOSS
                y_val_pred = model(X_val)
                val_loss = loss_fn(y_val_pred, Y_val)

                # SUM OF MASS FRACTION
                #Inverse scale done by hand to stay with Torch arrays
                yk = Yscaler_mean + (Yscaler_std + 1e-7)*y_val_pred
                if log_transform:
                    yk = torch.exp(yk)
                sum_yk = yk.sum(axis=1)
                sum_yk = sum_yk.detach().cpu().numpy()
                stats_sum_yk[epoch//10,0] = sum_yk.mean() 
                stats_sum_yk[epoch//10,1] = sum_yk.min()
                stats_sum_yk[epoch//10,2] = sum_yk.max()

                # ELEMENTS CONSERVATION
                yval_in = Xscaler_mean[1:] + (Xscaler_std[1:] + 1e-7)*X_val[:,1:]
                if log_transform:
                    yval_in = torch.exp(yval_in)
                ye_in = torch.matmul(A_element, torch.transpose(yval_in, 0, 1))
                ye_out = torch.matmul(A_element, torch.transpose(yk, 0, 1))
                delta_ye = (ye_out - ye_in)/(ye_in+1e-10)
                delta_ye = delta_ye.detach().cpu().numpy()
                stats_A_elements[epoch//10, :, 0] = delta_ye.mean(axis=1)
                stats_A_elements[epoch//10, :, 1] = delta_ye.min(axis=1)
                stats_A_elements[epoch//10, :, 2] = delta_ye.max(axis=1)

            model.train()   # Back to training mode
            val_loss_list[epoch//10] = val_loss

        print(f"Finished epoch {epoch}")
        print(f"    >> Loss: {loss}")
        if epoch%10==0:
            print(f"    >> Validation loss: {val_loss}")

    return epochs, epochs_small, loss_list, val_loss_list, stats_sum_yk, stats_A_elements

Routine to plot training & validation errors:

In [ ]:
def plot_losses_conservation(model_folder_i, epochs, epochs_small, loss_list, val_loss_list, stats_sum_yk, stats_A_elements):

    # LOSSES
    fig, ax = plt.subplots()

    ax.plot(epochs, loss_list, color="k", label="Training")
    ax.plot(epochs_small, val_loss_list, color="r", label = "Validation")

    ax.set_yscale('log')

    ax.legend()

    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")

    fig.tight_layout()

    fig.savefig(os.path.join(model_folder_i, "loss.png"))

    plt.close()

    # MASS CONSERVATION
    fig, ax = plt.subplots()

    ax.plot(epochs_small, stats_sum_yk[:,0], color="k")
    ax.plot(epochs_small, stats_sum_yk[:,1], color="k", ls="--")
    ax.plot(epochs_small, stats_sum_yk[:,2], color="k", ls="--")

    ax.set_xlabel("Epoch")
    ax.set_ylabel(r"$\sum_k \ Y_k$")

    fig.tight_layout()

    fig.savefig(os.path.join(model_folder_i, "sumYk.png"))

    plt.close()

    # ELEMENTS CONSERVATION
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)

    # C
    ax1.plot(epochs_small, 100*stats_A_elements[:,0,0], color="k")
    ax1.plot(epochs_small, 100*stats_A_elements[:,0,1], color="k", ls="--")
    ax1.plot(epochs_small, 100*stats_A_elements[:,0,2], color="k", ls="--")

    ax1.set_xlabel("Epoch")
    ax1.set_ylabel(r"$\Delta Y_C$ $(\%$)")

    # H
    ax2.plot(epochs_small, 100*stats_A_elements[:,1,0], color="k")
    ax2.plot(epochs_small, 100*stats_A_elements[:,1,1], color="k", ls="--")
    ax2.plot(epochs_small, 100*stats_A_elements[:,1,2], color="k", ls="--")

    ax2.set_xlabel("Epoch")
    ax2.set_ylabel(r"$\Delta Y_H$ $(\%)$")

    # O
    ax3.plot(epochs_small, 100*stats_A_elements[:,2,0], color="k")
    ax3.plot(epochs_small, 100*stats_A_elements[:,2,1], color="k", ls="--")
    ax3.plot(epochs_small, 100*stats_A_elements[:,2,2], color="k", ls="--")

    ax3.set_xlabel("Epoch")
    ax3.set_ylabel(r"$\Delta Y_O$ $(\%)$")

    # N
    ax4.plot(epochs_small, 100*stats_A_elements[:,3,0], color="k")
    ax4.plot(epochs_small, 100*stats_A_elements[:,3,1], color="k", ls="--")
    ax4.plot(epochs_small, 100*stats_A_elements[:,3,2], color="k", ls="--")

    ax4.set_xlabel("Epoch")
    ax4.set_ylabel(r"$\Delta Y_N$ $(\%)$")

    fig.tight_layout()

    fig.savefig(os.path.join(model_folder_i, "elements.png"))

    plt.close()

### Training hyperparameters

The hyper-parameters are given in lists. Here we set everything manually, we need thus to know in advance the number of clusters and their size.

In [ ]:
n_epochs_list = [1000, 1000, 200, 200]
batch_size_list = [256, 256, 256, 256]

In [ ]:
class ChemNN_1(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(n_in, 60)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(60, 60)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(60, n_out)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.output(x)
        return x
    

class ChemNN_2(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(n_in, 60)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(60, 60)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(60, n_out)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.output(x)
        return x
    

class ChemNN_3(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(n_in, 60)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(60, 60)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(60, n_out)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.output(x)
        return x
    

class ChemNN_4(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(n_in, 60)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(60, 60)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(60, n_out)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.output(x)
        return x
    

model_list = [ChemNN_1(), ChemNN_2(), ChemNN_3(), ChemNN_4()]

for model in model_list:
    model = model.to(device)

### Main training loop on clusters

Creating main folder to store models:

In [ ]:
model_folder = os.path.join(folder, "ann_models")
if not os.path.isdir(model_folder):
    os.mkdir(model_folder)

In [ ]:
for i_cluster in range(n_clusters):

    print("-------------------------------------------")
    print(f"         TRAINING MODEL {i_cluster}       ")
    print("-------------------------------------------")

    model_folder_i = os.path.join(folder, f"ann_models/cluster_{i_cluster}")
    if not os.path.isdir(model_folder_i):
        os.mkdir(model_folder_i)


    # Cluster hyperparameters
    model = model_list[i_cluster]
    n_epochs = n_epochs_list[i_cluster]
    batch_size = batch_size_list[i_cluster]
    
    # Load the data
    Xscaler, Yscaler, X_train, X_val, Y_train, Y_val = load_data(i_cluster)

    X_train = torch.tensor(X_train.values, dtype=torch.float64)
    Y_train = torch.tensor(Y_train.values, dtype=torch.float64)
    X_val = torch.tensor(X_val.values, dtype=torch.float64)
    Y_val = torch.tensor(Y_val.values, dtype=torch.float64)

    # Converting to torch arrays
    Xscaler_mean = torch.from_numpy(Xscaler.mean.values)
    Xscaler_std = torch.from_numpy(Xscaler.std.values)

    Yscaler_mean = torch.from_numpy(Yscaler.mean.values)
    Yscaler_std = torch.from_numpy(Yscaler.std.values)

    # Moving to correct device
    X_train = X_train.to(device)
    Y_train = Y_train.to(device)
    X_val = X_val.to(device)
    Y_val = Y_val.to(device)

    Xscaler_mean = Xscaler_mean.to(device)
    Xscaler_std = Xscaler_std.to(device)

    Yscaler_mean = Yscaler_mean.to(device)
    Yscaler_std = Yscaler_std.to(device)

    # Loss 
    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    epochs, epochs_small, loss_list, val_loss_list, stats_sum_yk, stats_A_elements = main_training_loop(X_train, X_val, Y_train, Y_val, loss_fn, optimizer, 
                                                                                                        n_epochs, batch_size, model, log_transform, 
                                                                                                        Xscaler_mean, Yscaler_mean, Xscaler_std, Yscaler_std)


    # Plotting losses and conservation metrics
    plot_losses_conservation(model_folder_i, epochs, epochs_small, loss_list, val_loss_list, stats_sum_yk, stats_A_elements)

    # Saving model
    torch.save(model.state_dict(), os.path.join(model_folder_i,"pytorch_mlp.pt"))                                                                                                
    

## ANN models testing

Loading k-means algorithm and scaler:

In [ ]:
with open(os.path.join(folder, "processed_database_cluster/kmeans_model.pkl"), "rb") as f:
    kmeans = pickle.load(f)

kmeans_scaler = joblib.load(os.path.join(folder, "processed_database_cluster/Xscaler_kmeans.pkl"))

Function to plot individual functions:

In [ ]:
def plot_results_sim_0D(i_sim, list_test_results, spec_to_plot):

    df_exact = list_test_results[i_sim][0]
    df_nn = list_test_results[i_sim][1]

    # Temperature 
    fig, ax = plt.subplots()

    ax.plot(df_exact['Time'], df_exact['Temperature'], color='k')
    ax.plot(df_nn['Time'], df_nn['Temperature'], color='b')
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("T [K]")

    # Species (normal)
    fig, ax = plt.subplots()

    ax.plot(df_exact['Time'], df_exact[spec_to_plot], color='k')
    ax.plot(df_nn['Time'], df_nn[spec_to_plot], color='b')
    ax.set_xlabel("Time [s]")
    ax.set_ylabel(f"{spec_to_plot} [-]")

    # Species (log)
    fig, ax = plt.subplots()

    ax.plot(df_exact['Time'], np.log(df_exact[spec_to_plot]), color='k')
    ax.plot(df_nn['Time'], np.log(df_nn[spec_to_plot]), color='b')
    ax.set_xlabel("Time [s]")
    ax.set_ylabel(f"{spec_to_plot} [-]")

    # Sum of Yk
    fig, ax = plt.subplots()
    ax.plot(df_nn['Time'], df_nn['SumYk'], color='b')
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("$\sum Y_k$ [-]")

    # Elements
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)
    ax1.plot(df_nn['Time'], df_nn['Y_C'], color='b')
    ax2.plot(df_nn['Time'], df_nn['Y_H'], color='b')
    ax3.plot(df_nn['Time'], df_nn['Y_O'], color='b')
    ax4.plot(df_nn['Time'], df_nn['Y_N'], color='b')
    ax1.set_ylabel("$Y_C$")
    ax2.set_ylabel("$Y_H$")
    ax3.set_ylabel("$Y_O$")
    ax4.set_ylabel("$Y_N$")
    ax3.set_xlabel("Time [s]")
    ax4.set_xlabel("Time [s]")
    fig.tight_layout()

In [ ]:
def plot_results_sim_1D(i_sim, list_test_results, spec_to_plot):

    df_exact = list_test_results[i_sim][0]
    df_nn = list_test_results[i_sim][1]

    # Species (normal)
    fig, ax = plt.subplots()

    ax.plot(df_exact['X'], df_exact[spec_to_plot], color='k')
    ax.plot(df_nn['X'], df_nn[spec_to_plot], color='b')
    ax.set_xlabel("x [m]")
    ax.set_ylabel(f"{spec_to_plot} [-]")

    # Sum of Yk
    fig, ax = plt.subplots()
    ax.plot(df_nn['X'], df_nn['SumOmegak'], color='b')
    ax.set_xlabel("x [m]")
    ax.set_ylabel("$\sum \dot{\omega}_k$ [-]")

    # Elements
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)
    ax1.plot(df_nn['X'], df_nn['Omega_C'], color='b')
    ax2.plot(df_nn['X'], df_nn['Omega_H'], color='b')
    ax3.plot(df_nn['X'], df_nn['Omega_O'], color='b')
    ax4.plot(df_nn['X'], df_nn['Omega_N'], color='b')
    ax1.set_ylabel("$\dot{\omega}_C$")
    ax2.set_ylabel("$\dot{\omega}_H$")
    ax3.set_ylabel("$\dot{\omega}_O$")
    ax4.set_ylabel("$\dot{\omega}_N$")
    ax3.set_xlabel("x [m]")
    ax4.set_xlabel("x [m]")
    fig.tight_layout()

### Testing on 0D reactors

We first load the test initial conditions:

In [ ]:
df_sim_test_0D = pd.read_csv(os.path.join(folder, "sim_test_0D.csv"))

n_sim_0D = df_sim_test_0D.shape[0]
print(f"There are {n_sim_0D} 0D test simulations")

In [ ]:
list_test_results_0D = []

fails = 0
for i, row in df_sim_test_0D.iterrows():

    phi_ini = row['Phi']
    temperature_ini = row['T0']

    print(f"Performing test computation for phi={phi_ini}; T0={temperature_ini}")

    df_exact, df_nn, fail = compute_nn_cantera_0D_homo(device, kmeans, kmeans_scaler, model_list, Xscaler, Yscaler, phi_ini, temperature_ini, dt, dtb_params, A_element.detach().cpu().numpy())

    fails += fail

    list_test_results_0D.append((df_exact, df_nn))


print(f"Total number of simulations which crashed: {fails}")

In [ ]:
i_sim = 9
spec_to_plot = "H2O2"
plot_results_sim_0D(i_sim, list_test_results_0D, spec_to_plot)

### Testing on 1D freely propagating premixed flames

In [ ]:
df_sim_test_1D_prem = pd.read_csv(os.path.join(folder, "sim_test_1D_prem.csv"))

n_sim_1D_prem = df_sim_test_1D_prem.shape[0]
print(f"There are {n_sim_1D_prem} 1D premixed test simulations")

In [ ]:
list_test_results_1D_prem = []

for i, row in df_sim_test_1D_prem.iterrows():

    phi_ini = row['Phi']
    temperature_ini = row['T0']

    print(f"Performing test computation for phi={phi_ini}; T0={temperature_ini}")

    df_exact, df_nn = compute_nn_cantera_1D_prem(device, kmeans, kmeans_scaler, model_list, Xscaler, Yscaler, phi_ini, temperature_ini, dt, dtb_params, A_element.detach().cpu().numpy())

    list_test_results_1D_prem.append((df_exact, df_nn))


In [ ]:
i_sim = 9
spec_to_plot = "OH"
plot_results_sim_1D(i_sim, list_test_results_1D_prem, spec_to_plot)

### Testing on 1D counterflow diffusion flames

In [ ]:
df_sim_test_1D_diff = pd.read_csv(os.path.join(folder, "sim_test_1D_diff.csv"))

n_sim_1D_diff = df_sim_test_1D_diff.shape[0]
print(f"There are {n_sim_1D_diff} 1D CF diffusion test simulations")

In [ ]:
list_test_results_1D_diff = []

width = 0.02

for i, row in df_sim_test_1D_diff.iterrows():

    strain = row['Strain']
    T0 = row['T0']

    print(f"Performing test computation for Strain={strain}; T0={T0}")

    df_exact, df_nn = compute_nn_cantera_1D_diff(device, kmeans, kmeans_scaler, model_list, Xscaler, Yscaler, strain, T0, width, dt, dtb_params, A_element.detach().cpu().numpy())

    list_test_results_1D_diff.append((df_exact, df_nn))


In [ ]:
i_sim = 10
spec_to_plot = "H2O"
plot_results_sim_1D(i_sim, list_test_results_1D_diff, spec_to_plot)